In [3]:
import os
import sys
import time
import json
import pprint
import logging

start = time.time()
pp = pprint.PrettyPrinter(indent = 2)
logger = logging.getLogger("reddit")

with open("../creds.json", "r") as g:
    conf = json.load(g)
    for k in conf.keys():
        print("key: {}".format(k))
    
    

key: twitter-api-key
key: twitter-secret-key
key: twitter-bearer
key: twitter-access-token
key: twitter-secret-access
key: aws-access-key
key: aws-secret-access-key


In [9]:
import boto3
import pyspark

secretmanager = boto3.client("secretsmanager")

subreddit = "technology"
spark_host = "spark-master" 
extra_jar_list = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2,org.apache.hadoop:hadoop-common:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-client:3.3.4,io.delta:delta-core_2.12:2.2.0,org.postgresql:postgresql:42.5.0"
aws_client = secretmanager.getSecretValue("AWS_ACCESS_KEY_ID")
aws_secret = secretmanager.getSecretValue("AWS_SECRET_ACCESS_KEY")

spark = pyspark.sql.SparkSession.builder \
                .master("spark://{}:7077".format(spark_host)) \
                .config("spark.scheduler.mode", "FAIR") \
                .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                .config("spark.executor.memory", "2048m") \
                .config("spark.executor.cores", "2") \
                .config("spark.streaming.concurrentJobs", "8") \
                .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                .config("spark.eventLog.enabled", "true") \
                .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
                .config("spark.sql.debug.maxToStringFields", 1000) \
                .config("spark.jars.packages", extra_jar_list) \
                .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                .enableHiveSupport() \
                .getOrCreate()

logger.info("created spark session.")

AttributeError: 'SecretsManager' object has no attribute 'getSecretValue'